# Unicode Problems & Answers

> This notebook documents answers to specific Unicode questions as part of the project.

## Problem (unicode1): Understanding Unicode

*(Answers to be filled in here)*

## Problem (unicode2): Unicode Encodings

*(Answers to be filled in here)*